In [3]:
# Importamos la biblioteca 'requests' para realizar solicitudes HTTP
import requests  # Permite enviar peticiones HTTP a un servidor

# Importamos 'BeautifulSoup' desde 'bs4' para parsear y navegar por el contenido HTML
from bs4 import BeautifulSoup  # Facilita la extracción de información de documentos HTML

# Importamos 'pandas' para manipular y analizar datos en estructuras de tipo DataFrame
import pandas as pd  # Ofrece herramientas para trabajar con datos estructurados

# Importamos 'matplotlib.pyplot' para generar gráficos básicos
import matplotlib.pyplot as plt  # Permite crear gráficos y visualizaciones

# Importamos 'seaborn' para generar visualizaciones estadísticas más avanzadas sobre matplotlib
import seaborn as sns  # Ofrece gráficos de mayor calidad y personalización

# Configuramos matplotlib para que los gráficos se muestren directamente en el Notebook.
%matplotlib inline

# Confirmamos que todas las librerías se han importado correctamente
print("Librerías importadas correctamente.")


Librerías importadas correctamente.
